In [8]:
from flask import Flask, jsonify
import threading

app = Flask(__name__)

@app.route('/api/greet/<name>', methods=['GET'])
def greet(name):
    return jsonify(message=f"Hello, {name}!")

# To get it to work in Colab, use this line instead of the usual app.run
threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':5000}).start()

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\bayram\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Test Cases for the Flask Application

#### Test Case: Valid Name Input

In [16]:
# When using the threading line above in Colab, you can now call your API in colab too!
import requests
req = requests.get("http://127.0.0.1:5000/api/greet/mutabay")
print(req.status_code)
print(req.text)

200
{
  "message": "Hello, mutabay!"
}


In [14]:
import unittest
from flask.testing import FlaskClient

class FlaskTestClient(FlaskClient):
    def open(self, *args, **kwargs):
        kwargs.setdefault('follow_redirects', True)
        return super().open(*args, **kwargs)

app.test_client_class = FlaskTestClient

class TestFlaskAPI(unittest.TestCase):
    def setUp(self):
        self.app = app.test_client()
        self.app.testing = True

    def test_basic_greeting(self):
        response = self.app.get('/api/greet/John')
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json, {"message": "Hello, John!"})

    def test_url_encoding(self):
        response = self.app.get('/api/greet/John%20Doe')
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json, {"message": "Hello, John Doe!"})

    def test_special_characters(self):
        response = self.app.get('/api/greet/@!$*')
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json, {"message": "Hello, @!$*!"})

    def test_empty_name(self):
        response = self.app.get('/api/greet/')
        self.assertEqual(response.status_code, 404)

    def test_numeric_name(self):
        response = self.app.get('/api/greet/12345')
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json, {"message": "Hello, 12345!"})

    def test_long_name(self):
        long_name = "a" * 1000
        response = self.app.get(f'/api/greet/{long_name}')
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json, {"message": f"Hello, {long_name}!"})

    def test_html_injection(self):
        html_name = "<script>alert('test')</script>"
        response = self.app.get(f'/api/greet/{html_name}')
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json, {"message": f"Hello, {html_name}!"})

    def test_json_response_format(self):
        response = self.app.get('/api/greet/John')
        self.assertEqual(response.headers['Content-Type'], 'application/json')
        self.assertTrue(response.json)

if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test_basic_greeting (__main__.TestFlaskAPI.test_basic_greeting) ... ok
test_empty_name (__main__.TestFlaskAPI.test_empty_name) ... ok
test_html_injection (__main__.TestFlaskAPI.test_html_injection) ... FAIL
test_json_response_format (__main__.TestFlaskAPI.test_json_response_format) ... ok
test_long_name (__main__.TestFlaskAPI.test_long_name) ... ok
test_numeric_name (__main__.TestFlaskAPI.test_numeric_name) ... ok
test_special_characters (__main__.TestFlaskAPI.test_special_characters) ... ok
test_url_encoding (__main__.TestFlaskAPI.test_url_encoding) ... ok

FAIL: test_html_injection (__main__.TestFlaskAPI.test_html_injection)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\bayram\AppData\Local\Temp\ipykernel_22096\2054321482.py", line 49, in test_html_injection
    self.assertEqual(response.status_code, 200)
AssertionError: 404 != 200

----------------------------------------------------------------------
Ran 8